In [36]:
import librosa
import os
from matplotlib import pyplot as plt
import IPython.display as ipd
import pandas as pd
from pydub import AudioSegment
import ffprobe
import ffmpeg
import numpy as np
import wave
import os
from pydub import AudioSegment
import numpy as np
from teager_py import Teager
import pydub

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [40]:
import zipfile

# zip file handler  
zip = zipfile.ZipFile('/home/xuranzeng/thesis/ReleasedDataset.zip')

# list available files in the container
filename = zip.namelist()

# clean path and file names
audioname = (item for item in filename if item.endswith('mp3')) 
data = pd.DataFrame(list(audioname))
data = pd.concat([data, data[0].str.split('/', 4, expand=True)], axis=1) # split path 
data = pd.concat([data, data[1].str.split('_', 2, expand=True)], axis=1) # split company and day
data.columns=["Path","Basic", "Company_Day", "Type", "Filename","Company","Day"] # rename column
data = data[['Path','Company',"Day","Filename"]] # delete useless columns

# path
base_path='/home/xuranzeng/thesis/EC/'
data['Complete_Path'] = base_path + data['Path'].astype(str)
data = data[['Complete_Path','Company',"Day","Filename"]] # delete useless columns

In [41]:
data

,Complete_Path,Company,Day,Filename
0,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_15_1.mp3
1,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_16_1.mp3
2,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_1_14.mp3
3,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_2_11.mp3
4,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_10_5.mp3
...,...,...,...,...
89838,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Lilly (Eli) & Co.,20170425,David A. Ricks_1_6.mp3
89839,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Lilly (Eli) & Co.,20170425,David A. Ricks_3_4.mp3
89840,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Lilly (Eli) & Co.,20170425,David A. Ricks_5_10.mp3
89841,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Lilly (Eli) & Co.,20170425,David A. Ricks_1_30.mp3


In [42]:
#data.to_csv('/home/xuranzeng/thesis/EC_path.csv')

In [ ]:
# feature extraction

In [79]:
# TEO
from teo_cb_auto_env import TeagorExtractor

import pydub
import librosa
from scipy import signal
from mutagen.mp3 import MP3
import numpy as np
from numpy import sign, trapz
from teager_py import Teager
import statsmodels.api as sm
import wave
import statistics
from pydub import AudioSegment

t = TeagorExtractor()

In [95]:
company_name = "3M Company"
day_name = "20170425"
sample = data.loc[(data["Company"]==company_name) & (data["Day"]==day_name)].reset_index(drop=True)

In [165]:
sample.head(2)

,Complete_Path,Company,Day,Filename,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,band11,band12,band13,band14,band15,band16
0,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,3M Company,20170425,Nicholas C. Gangestad_9_3.mp3,22.707478,25.386090,29.092349,31.91716,27.745593,25.787118,25.252590,23.376251,25.050941,24.627595,24.042610,24.736106,25.619664,29.304457,30.982242,21.329632
1,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,3M Company,20170425,Nicholas C. Gangestad_1_37.mp3,23.248017,25.966949,28.807318,31.86898,27.904177,25.612565,25.447218,23.700010,24.866343,24.927017,24.778911,25.174092,25.126332,28.153340,28.744018,21.458480


In [162]:
def get_teo(sample):
    TEO = []
    for i in range(0,len(sample)):
        filepath = sample.iloc[i,0]
        teo_cb_auto_env = t.process_mp3(filepath,1)
        TEO.append(pd.DataFrame(pd.DataFrame(teo_cb_auto_env).mean(axis=0)).T)
    #print('processing:',i)
    TEO = pd.concat(TEO).reset_index()
    return TEO


In [163]:
TE0_0105 = get_teo(sample)

Computing TEO Features ...
0 2
Computing TEO Features ...
0 5


In [101]:
#TE0_0105.to_csv('/home/xuranzeng/thesis/TEO_0105.csv')

In [ ]:
# GeMAPS
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

In [131]:
def get_GeMAPS(path):
    GeMAPS = []
    for i in range(0,len(path)):
        filepath = path.loc[i,'Complete_Path']
        audio = AudioSegment.from_mp3(filepath)
        sr = audio.frame_rate
        audio = audio.get_array_of_samples()
        df = smile.process_signal(audio,sr)
        GeMAPS.append(pd.DataFrame(df.mean(axis=0)).T)
        #print('processing:',i)
    

    GeMAPS = pd.concat(GeMAPS).reset_index()
    return GeMAPS

In [137]:
#GeMAPS_0105.to_csv('/home/xuranzeng/thesis/GeMAPS_0105.csv')

In [ ]:
GeMAPS_0105 = get_GeMAPS(sample)

In [145]:
features = pd.concat([sample, GeMAPS_0105], axis=1)

In [147]:
#features.to_csv('/home/xuranzeng/thesis/features_0105.csv')